In [1]:
import pandas as pd
import numpy as np
import requests
import os
import yaml
import gzip
import random

In [5]:
url= "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
filename= url.split("/")[-1].replace('.',"_")
filepath= os.path.join(os.getcwd(),'data/',filename+'.gz')

with open(filepath, "wb") as f:
    r = requests.get(url)
    f.write(r.content)
    f.close()

In [3]:
df= pd.read_csv('/app/data/amazon_reviews_us_Beauty_v1_00_tsv_gz.gz',compression='gzip',index_col=None,header=0, sep='\t', lineterminator='\r')
print(len(df))
print(df.size)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [2]:
from extractor import DataExtractor

filename = DataExtractor().get_data(sample_size=0.01)
filename

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,23817168,R5ERURA8D91R5,B00LNOV8JO,215421233,Radha Beauty Rosehip Oil 4 oz - USDA Certified...,Beauty,5,0,0,N,N,i like it,This stuff is pretty good ND the age 4oz make ...,2015-08-31\n
1,US,12728480,R26NLYA8OEC3ZR,B0027VCNGO,24106546,Seche Vite Dry Fast Top Coat High Shine Profes...,Beauty,5,0,1,N,Y,Best top coat I ever try,The best top coat I ever try,2015-08-30\n
2,US,38234991,R38838I8YV70TI,B0082D6C2M,41461972,"Revlon Color Stay 24 Mascara, 0.21 Fluid Ounce",Beauty,5,1,1,N,Y,Excellent for days you'll be by the water or w...,It's waterproof and wont come off without make...,2015-08-29\n
3,US,39225039,RSRG93O1D7D8G,B00I9KX1OU,274108821,"Energizer EBR1225 (BR1225, CR1225) Lithium Coi...",Beauty,5,0,0,N,Y,Five Stars,Good buy,2015-08-23\n
4,US,36159796,R24MT495I9Q9FH,B008BUKUFQ,778845866,Outre Premium Duby Human Hair Wig DUBY,Beauty,1,0,1,N,Y,One Star,"Do not buy, wig is not natural.",2015-08-22\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,US,36013545,R2IY5TSCVXK037,B000C214LA,46834005,Quasar Cologne by J. Del Pozo for men Colognes,Beauty,5,4,4,N,N,I love it !,I was given this Eau De Toilette a few years a...,2008-04-17\n
485,US,32437204,R2S3G9CR0COMHI,B0009V1YS2,642087527,Farouk CHI GF1539 Turbo Big 2 Inch Ceramic Fla...,Beauty,3,7,7,N,N,Warranty,Bought this for my fiance. It lasted 9 months...,2008-04-01\n
486,US,25143359,R1N4UKE3OB74XH,B0007UD8FC,106287948,Clarins True Radiance Foundation,Beauty,4,0,0,N,N,Always looking.,When I first started using this product I was ...,2007-10-06\n
487,US,28004080,R2YPYQG041SESJ,B00008V408,553825360,Conair HBRPO8 Hair Buffer Replacement Pads (Pa...,Beauty,4,0,0,N,Y,It would get 5 stars if it was smaller,"great, leaves my skin smooth soft, doesn't sca...",2006-11-11\n


In [4]:
filename['star_rating'].max()

'5'

In [30]:
url= "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
filename= url.split("/")[-1].replace('.',"_")
filepath= os.path.join(os.getcwd(),'data/',filename+'.gz')

header_filepath = os.path.join(os.getcwd(),'data/header.csv')
train_pos_filepath = os.path.join(os.getcwd(),'data/train_pos.csv')
train_neg_filepath = os.path.join(os.getcwd(),'data/train_neg.csv')
test_pos_filepath = os.path.join(os.getcwd(),'data/test_pos.csv')
test_neg_filepath = os.path.join(os.getcwd(),'data/test_neg.csv')


combined_filepath = [header_filepath, train_pos_filepath, train_neg_filepath, test_pos_filepath, test_neg_filepath]


def split_train_test_samples(sample_size: float, seed: int) -> pd.DataFrame:
    """
    creates a dataframe of random sample from the file
    :param sample_size: probability of selecting a sample and should be between 0 and 100
    :param seed: This is to assign random seed for reproducability
    :return: returns a pandas dataframe
    """
    #if sample_size<0 | sample_size> 100:
     #   raise Exception("Sample size need to be between 0 and 100")
    
    for path in combined_filepath:
        if os.path.exists(path):
            os.remove(path)
    
    with gzip.open(filepath, 'rt', encoding="utf8") as f, \
        open(header_filepath,'w') as header, \
        open(train_pos_filepath,'w') as train_pos, \
        open(train_neg_filepath,'w') as train_neg, \
        open(test_pos_filepath,'w') as test_pos, \
        open(test_neg_filepath,'w') as test_neg:
            
        header.write(f.readline())
        random.seed(seed)
        for line in f:
            sample = random.choices([0, 1], weights=[100 - sample_size, sample_size], k=1)
            fields = line.split('\t')
            polarity = -1 if int(fields[7]) <3 else 1 if int(fields[7]) >3 else 0
            if sample[0] == 1:
                if polarity == 1:
                    train_pos.write(line)
                elif polarity == -1:
                    train_neg.write(line)
            else:
                if polarity == 1:
                    test_pos.write(line)
                elif polarity == -1:
                    test_neg.write(line)

split_train_test_samples(70, 123)

In [17]:
pd.read_csv(train_filepath,sep="\t", header=None)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,US,2126364,R1H2WWD1OB5ERC,B004LLIL5A,346014806,Amazon eGift Card - Celebrate,Gift Card,5,0,0,N,Y,Five Stars,Excelente,2015-08-29
1,US,23139302,R3RII08BG7QTJ4,B004KNWWO0,802996579,Amazon.com Gift Cards - Print at Home,Gift Card,5,0,0,N,Y,Five Stars,great product,2015-05-19
2,US,42476645,R3V0NH6I258ZEQ,B0080IR4MQ,676326750,Amazon.com Gift Card in a Brown & Blue Tin (Cl...,Gift Card,2,1,2,N,Y,"Gift card is a gift card, and I know ...","Gift card is a gift card, and I know it was fr...",2015-04-16
3,US,32046574,R3ID6NSVRA94JH,B004LLIKVU,473048287,Amazon.com eGift Cards,Gift Card,5,0,0,N,Y,Fino,Fino,2014-09-03
4,US,17608529,RQ0Y9DKSL26IY,B004LLIL6Y,799327567,Amazon eGift Card - New Baby (Monkey),Gift Card,5,0,0,N,Y,GREAT Product,I was VERY happy to be able to purchase an onl...,2014-05-28
5,US,25891611,R15ON2L9ZNRHBC,B0062ONJRE,334262000,Amazon Gift Card - Print - Merry Christmas (Na...,Gift Card,4,0,0,N,Y,quick download,This was a gift for my daughter and her new ki...,2014-01-30
6,US,11822098,R2SDJQLAAMP18O,B00A4EK3UE,973029669,Amazon eGift Card - Birthday (Fonts),Gift Card,5,0,0,N,Y,Quick and Easy Gifts,If you are like me with lots of family members...,2013-07-10
7,US,13116241,R1LSVGSKZ4W9SD,B005J0423C,775486538,Amazon.com Gift Card in a Greeting Card (Vario...,Gift Card,5,0,0,N,Y,Everyone loves a Gift Card!,"Arrived quickly, made the niece really happy a...",2013-06-17
8,US,28828709,R1XR7VU1HEFNU6,B00A4EK2FK,576358674,Amazon Gift Card - Print - Seasonal (Happy Spr...,Gift Card,5,0,0,N,Y,Gift card,Love getting gift cards and giving them! This ...,2013-05-20
9,US,26864097,R1PSQIG7HJIIR1,B004KNWWMW,647374203,Amazon Gift Card - Print - Happy Birthday (Bal...,Gift Card,5,0,0,N,Y,Great product!!,Would purchase one of these giftcards again. ...,2013-03-17


In [1]:
from utils.utils import TextProcessing

text = "this is america and its beautiful."
TextProcessing().process_sentence(sentence=text)

['america', 'beautiful', '.']

In [15]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


In [24]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
word_clean = []
for word in tok:
    word_clean.extend(lemmatizer(word))
word